# Inaugural Project

> **Note the following:** 
> 1. This is an example of how to structure your **inaugural project**.
> 1. Remember the general advice on structuring and commenting your code
> 1. The `inauguralproject.py` file includes a function which can be used multiple times in this notebook.

Imports and set magics:

In [1]:
import numpy as np

# autoreload modules when code is run. Otherwise, python will not see recent changes. 
%load_ext autoreload
%autoreload 2

# Import your own code
import matplotlib.pyplot as plt
from HouseholdSpecializationModel import HouseholdSpecializationModelClass

# Question 1

**Explain how you solve the model**

In [3]:
#trying to solve the first question : 

# create an instance of the model class
model = HouseholdSpecializationModelClass()

# define the values of alpha and sigma to loop over
alpha_vals = [0.25, 0.5, 0.75]
sigma_vals = [0.5, 1.0, 1.5]

# create a figure with subplots for each combination of alpha and sigma
fig, axs = plt.subplots(len(alpha_vals), len(sigma_vals), figsize=(10, 8), sharex=True, sharey=True)

# loop over each combination of alpha and sigma
for i, alpha in enumerate(alpha_vals):
    for j, sigma in enumerate(sigma_vals):
        # set the alpha and sigma parameters of the model
        model.par.alpha = alpha
        model.par.sigma = sigma

        # solve the model discretely
        opt = model.solve_discrete()

        # calculate the ratio of HF/HM
        ratio = opt.HF / opt.HM

        # plot the ratio against wF
        axs[i,j].plot(model.par.wF_vec, ratio)
        axs[i,j].set_title(f"alpha={alpha}, sigma={sigma}")
        axs[i,j].set_ylim([0, 4])

# add axis labels and title
fig.text(0.5, 0.04, 'Female wage (wF)', ha='center')
fig.text(0.04, 0.5, 'HF/HM', va='center', rotation='vertical')
fig.suptitle('Ratio of HF/HM for different values of alpha and sigma')

# adjust spacing between subplots
plt.subplots_adjust(wspace=0.1, hspace=0.3)

# show the plot
plt.show()




NameError: name 'x' is not defined

# Question 2

Explain your code and procedure

In [1]:
# code

Y = 123

X = 456

Z = 890

# Question 3

Explain your code and procedure

In [ ]:
# code

ADD CONCISE CONLUSION.